In [ ]:

# 2. X, y 정의
# 3. X, y 배치화


In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [86]:
# 1. 데이터 불러오는 거
data = pd.read_csv('/content/wine.csv')
data.head(2) # 잘 불러왔군

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1


In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  class                 6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [98]:
# 2. X, y 정의
features = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates']
target = 'alcohol'

X = data[features]
y = data[target]


In [99]:
from sklearn.model_selection import train_test_split
import numpy as np
# 8대2 쪼개기
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=13)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [100]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape,

((5197, 10), (1300, 10), (5197,), (1300,))

In [101]:
# 4, X, y를 배치화
from torch.utils.data import TensorDataset

X_train = torch.FloatTensor(X_train.values) # 이유 : 파이토치에서는 텐서타입을 사용!
y_train = torch.FloatTensor(y_train.values)
X_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(y_test.values)

train_set = TensorDataset(X_train, y_train) # X와 y를 짝꿍 만들어 배치로 짤름!
test_set = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_set, batch_size = 8, shuffle = True)
test_loader = DataLoader(test_set, batch_size = 8, shuffle = False)

In [103]:
# 4. 모델정의
import torch.nn. functional as F

class MLPRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 4) # 10개의 피쳐 입력 # 4개 아웃
        self.layer2 = nn.Linear(4, 1) # 4개 입력, 최종 출력 1개
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

model = MLPRegression()

In [104]:
# 테스트
model(next(iter(train_loader))[0])
#train_loader에서 첫번째 배치 꺼낸 다음 0번째 부분인 X만

tensor([[4.3656],
        [4.0550],
        [8.1029],
        [0.3808],
        [0.2761],
        [5.8490],
        [8.2423],
        [3.4827]], grad_fn=<AddmmBackward0>)

In [105]:
# 5. 옵티마이저, loss 정의
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)


In [106]:
# 7. 학습
epochs = 10

for i in range(epochs):
    for idx, data in enumerate(train_loader):
        X, y = data
        pred = model(X)
        loss = criterion(pred,y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch: {i} loss: {loss}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 0 loss: 9.276658058166504
epoch: 1 loss: 2.414743423461914
epoch: 2 loss: 0.9731122851371765
epoch: 3 loss: 2.5690009593963623
epoch: 4 loss: 1.2330979108810425
epoch: 5 loss: 3.6302249431610107
epoch: 6 loss: 1.4264976978302002
epoch: 7 loss: 1.7336130142211914
epoch: 8 loss: 5.0576653480529785
epoch: 9 loss: 1.9526176452636719


In [108]:
# 8. 평가
temp = []
for idx, data in enumerate(test_loader):
    X,y = data
    pred = model(X)
    loss = criterion(pred,y)
    temp.append(loss)

sum(temp)/len(temp)




tensor(1.6972, grad_fn=<DivBackward0>)